In [61]:
%reload_ext autoreload
%autoreload 2

In [62]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from yawning_titan.envs.generic.core.blue_interface import BlueInterface
from yawning_titan.envs.generic.core.network_interface import NetworkInterface
from yawning_titan.envs.generic.core.red_interface import RedInterface
from yawning_titan.envs.generic.generic_env import GenericNetworkEnv
from yawning_titan.game_modes.game_mode_db import default_game_mode
from yawning_titan.networks.network_db import default_18_node_network

In [63]:
game_mode = default_game_mode()

In [64]:
network = default_18_node_network()

In [65]:
network_interface = NetworkInterface(game_mode=game_mode, network=network)

In [66]:
red = RedInterface(network_interface)
blue = BlueInterface(network_interface)

In [67]:
env = GenericNetworkEnv(red, blue, network_interface)

In [72]:
check_env(env, warn=True)

_ = env.reset()

In [73]:
from aegis.random_agent import RandomAgent, RandomPolicy

agent = RandomAgent(env=env, policy=RandomPolicy)

In [74]:
rm, rstd = evaluate_policy(agent, Monitor(env), n_eval_episodes=100, return_episode_rewards=False)

In [75]:
print(f"Mean reward: {rm:.4f}\nStd Reward: {rstd:.4f}")

Mean reward: -128.5739
Std Reward: 26.0821
